In [1]:
import pandas as pd

# 수상횟수 종합

In [2]:
df1 = pd.read_excel('수상내역데이터/수상내역.xlsx', sheet_name=0)[['연도','수상자','생년월일']] # 정규시즌MVP
df2 = pd.read_excel('수상내역데이터/수상내역.xlsx', sheet_name=1)[['연도','수상자','생년월일']] # 포스트시즌MVP
df3 = pd.read_excel('수상내역데이터/수상내역.xlsx', sheet_name=2)[['연도','수상자','생년월일']] # 신인왕
df4 = pd.read_excel('수상내역데이터/수상내역.xlsx', sheet_name=3)[['연도','수상자','생년월일']] # 골든글러브

In [3]:
for YEAR in range(2002, 2021):
    
    # 연도별 데이터 찾아오기
    df1_year = df1[df1['연도']==YEAR]
    df2_year = df2[df2['연도']==YEAR]
    df3_year = df3[df3['연도']==YEAR]
    df4_year = df4[df4['연도']==YEAR]
    
    # 연도별 개수 파악
    df_year = pd.concat([df1_year, df2_year, df3_year, df4_year]).reset_index(drop=True)
    df_year = df_year.value_counts().reset_index().rename(columns = {0:'수상횟수'})

    # concat
    if YEAR == 2002:
        df_final = df_year.copy()
    else:
        df_final = pd.concat([df_final, df_year]).reset_index(drop=True)
        
df_final

,연도,수상자,생년월일,수상횟수
0,2002,마해영,1970-08-14,2
1,2002,이승엽,1976-08-18,2
2,2002,김종국,1973-09-14,1
3,2002,김한수,1971-10-30,1
4,2002,박용택,1979-04-21,1
...,...,...,...,...
246,2020,오재원,1985-02-09,1
247,2020,이정후,1998-08-20,1
248,2020,최형우,1983-12-16,1
249,2020,플렉센,1994-07-01,1


In [4]:
foreign_player = ['브리또', '로페즈', '브룸바', '데이비스', '서튼', '리오스', '가르시아', '나바로', '테임즈', 
                '밴헤켄', '니퍼트', '해커', '버나디나', '린드블럼', '로하스', '샌즈', '페르난데스', '알칸타라', '플렉센']

df_final = df_final[~df_final['수상자'].isin(foreign_player)].reset_index(drop=True)
df_final = df_final.rename(columns={'수상자':'선수명'})
df_final['생년월일'] = df_final['생년월일'].astype('str')
df_final

,연도,선수명,생년월일,수상횟수
0,2002,마해영,1970-08-14,2
1,2002,이승엽,1976-08-18,2
2,2002,김종국,1973-09-14,1
3,2002,김한수,1971-10-30,1
4,2002,박용택,1979-04-21,1
...,...,...,...,...
220,2020,소형준,2001-09-16,1
221,2020,오재원,1985-02-09,1
222,2020,이정후,1998-08-20,1
223,2020,최형우,1983-12-16,1


# Merge

In [5]:
# 데이터 불러오기
hitter = pd.read_csv('선수데이터(전처리완료)/타자.csv')
pitcher = pd.read_csv('선수데이터(전처리완료)/투수.csv')

hitter.shape, pitcher.shape

((4239, 66), (3557, 70))

In [6]:
# merge
hitter = pd.merge(hitter, df_final, on=['선수명', '생년월일', '연도'], how='left')
pitcher = pd.merge(pitcher, df_final, on=['선수명', '생년월일', '연도'], how='left')

hitter.shape, pitcher.shape

((4239, 67), (3557, 71))

In [7]:
# 합쳐진 데이터의 개수로 잘 합쳐졌는지 검산
hitter['수상횟수'].notnull().sum() + pitcher['수상횟수'].notnull().sum()

225

In [9]:
# 다른값들은 수상횟수 0으로 채운다.
hitter['수상횟수'] = hitter['수상횟수'].fillna(0)
pitcher['수상횟수'] = pitcher['수상횟수'].fillna(0)

In [10]:
hitter.to_csv('선수데이터(전처리완료)/타자.csv', index=False, encoding='utf-8-sig')
pitcher.to_csv('선수데이터(전처리완료)/투수.csv', index=False, encoding='utf-8-sig')